In [102]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from utils import process_audio_files, create_sequences
import os
import random
import pandas as pd

# Load the dataframe
df = pd.read_csv('output/data/merged_df.csv')

# Get the list of file paths from the dataframe
file_paths = df['file_path'].tolist()

# Directories containing the files
gt_dir = './content/LibriSeVoc/gt'
diffwave_dir = './content/LibriSeVoc/diffwave'

# Get list of files in each directory
gt_files = [gt_dir + '/' + f for f in os.listdir(gt_dir) if os.path.join(gt_dir, f) not in file_paths]
diffwave_files = [diffwave_dir + '/' + f for f in os.listdir(diffwave_dir) if os.path.join(diffwave_dir, f) not in file_paths]

# Randomly select 10 files from each directory
selected_gt_files = random.sample(gt_files, 100)
selected_diffwave_files = random.sample(diffwave_files, 100)

print("Selected GT files:", selected_gt_files)
print("Selected Diffwave files:", selected_diffwave_files)

Selected GT files: ['./content/LibriSeVoc/gt/2182_181173_000018_000000.wav', './content/LibriSeVoc/gt/1841_179183_000015_000004.wav', './content/LibriSeVoc/gt/446_123502_000009_000001.wav', './content/LibriSeVoc/gt/3857_182317_000028_000001.wav', './content/LibriSeVoc/gt/412_126975_000006_000000.wav', './content/LibriSeVoc/gt/2136_5143_000015_000000.wav', './content/LibriSeVoc/gt/4640_19188_000052_000000.wav', './content/LibriSeVoc/gt/2136_5147_000050_000002.wav', './content/LibriSeVoc/gt/8312_279791_000016_000001.wav', './content/LibriSeVoc/gt/7505_258964_000011_000005.wav', './content/LibriSeVoc/gt/7078_271888_000114_000000.wav', './content/LibriSeVoc/gt/1098_133695_000006_000012.wav', './content/LibriSeVoc/gt/8468_294887_000023_000000.wav', './content/LibriSeVoc/gt/6147_34607_000013_000029.wav', './content/LibriSeVoc/gt/8630_305212_000004_000001.wav', './content/LibriSeVoc/gt/40_121026_000039_000000.wav', './content/LibriSeVoc/gt/405_130894_000057_000000.wav', './content/LibriSeVoc/

In [103]:
# Load the model
model = load_model('best_model.keras')
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_21 (LSTM)                  │ (None, 10, 64)         │        24,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 10, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 277,445 (1.06 MB)

 Trainable params: 92,481 (361.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 184,964 (722.52 KB)

In [104]:
# Create a DataFrame from selected_gt_files and selected_diffwave_files
selected_files_df = pd.DataFrame({
    'file_path': selected_gt_files + selected_diffwave_files,
    'label': [1] * len(selected_gt_files) + [0] * len(selected_diffwave_files)
})

selected_files_df.to_csv('output/data/selected_files.csv', index=False)

In [105]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


# Function to evaluate the model
def evaluate_model(model, audio_paths, labels):
    # features = process_audio_files(audio_paths, labels=labels, output_csv='test.csv')
    df_test = pd.read_csv('test.csv')

    # Extract features and labels from the dataframe
    features = df_test.drop(columns=['label', 'sampling_rate', 'file_path'])
    print(features.columns)
    labels = df_test['label']
    print('Test data:', features.shape, labels.shape)

    # 1. Create sequences
    sequence_length = 10  # Example value, adjust as needed
    overlap = 5  # Example value, adjust as needed
    sequences, indices = create_sequences(features, sequence_length, overlap)
    labels = labels[indices]

    # 2. Pad sequences
    padded_sequences = pad_sequences(
        sequences, maxlen=sequence_length, padding="pre", truncating="post"
    )

    # Scale features
    with open('./output/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    # Reshape padded_sequences for scaling
    num_samples, seq_len, num_features = padded_sequences.shape
    padded_sequences_reshaped = padded_sequences.reshape(
        num_samples, seq_len * num_features
    )

    # Fit and transform the scaler on the reshaped data
    padded_sequences_scaled = scaler.fit_transform(padded_sequences_reshaped)

    # Reshape the scaled data back to the original shape
    padded_sequences = padded_sequences_scaled.reshape(
        num_samples, seq_len, num_features
    )

    # Make predictions
    predictions = model.predict(padded_sequences)
    print(predictions)
    # Evaluate the model
    # accuracy = np.mean((np.array(predictions) >= 0.5).astype(int) == labels)
    # print(f"Model accuracy: {accuracy * 100:.2f}%")

# Create a list of audio paths and corresponding labels
audio_paths = selected_gt_files + selected_diffwave_files
labels = [1] * len(selected_gt_files) + [0] * len(selected_diffwave_files)

print(audio_paths)
print(labels)
evaluate_model(model, audio_paths=audio_paths, labels=labels)

['./content/LibriSeVoc/gt/2182_181173_000018_000000.wav', './content/LibriSeVoc/gt/1841_179183_000015_000004.wav', './content/LibriSeVoc/gt/446_123502_000009_000001.wav', './content/LibriSeVoc/gt/3857_182317_000028_000001.wav', './content/LibriSeVoc/gt/412_126975_000006_000000.wav', './content/LibriSeVoc/gt/2136_5143_000015_000000.wav', './content/LibriSeVoc/gt/4640_19188_000052_000000.wav', './content/LibriSeVoc/gt/2136_5147_000050_000002.wav', './content/LibriSeVoc/gt/8312_279791_000016_000001.wav', './content/LibriSeVoc/gt/7505_258964_000011_000005.wav', './content/LibriSeVoc/gt/7078_271888_000114_000000.wav', './content/LibriSeVoc/gt/1098_133695_000006_000012.wav', './content/LibriSeVoc/gt/8468_294887_000023_000000.wav', './content/LibriSeVoc/gt/6147_34607_000013_000029.wav', './content/LibriSeVoc/gt/8630_305212_000004_000001.wav', './content/LibriSeVoc/gt/40_121026_000039_000000.wav', './content/LibriSeVoc/gt/405_130894_000057_000000.wav', './content/LibriSeVoc/gt/7078_271888_0001